# Using Jurassic-1 Grande on SageMaker through Model Packages

This sample notebook shows you how to deploy Jurassic-1 Grande using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This noebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create an input prompt](#B.-Create-an-input-prompt)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Create your own CV Profile!](#D.-Create-your-own-CV-Profile!)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select to the model package
Confirm that you recieved this notebook from model catalog on SageMaker AWS Management Console.

In [10]:
# Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:416407187090:model-package/j1-grande-dbf0249",
#     "eu-west-1": "",
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

In [3]:
! pip install -U ai21-python-client[SM]
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.


In [4]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [5]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [6]:
model_name = "j1-grande"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create an input prompt

You can think of Jurassic-1 Grande as a smart auto-completion algorithm: it is very good at latching on to hints and patterns expressed in plain English, and generating text that follows the same patterns.
Here are two words you should know:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generated.

The best way to guide the model is to provide several examples, consisting of input-output pairs, in the prompt.
In order to help the model distinguish between each example, we use a stop sequence. The stop sequence itself should be immediately recognizeable when you look at the text as a whole.

In this example, we will create a CV profile, a short paragraph in a resume that highlights the candidate’s background, skills and ambitions.

The prompt comprised from the following:
1. Few-shot examples: pairs of input-output pairs. In this case, the input is a role and some skills to highlight in the CV profile. They are separated by the following stop sequence: ##.

2. A role and skills to create CV profile from. It should be written in the same format as all the examples.

The following function gets a new reviews and creates a few-shot prompt to send to Jurassic-1 Grande:

In [9]:
def create_cv_profile_prompt(role, skills):
    """
    Given a role and skill highlights, create a prompt for Jurassic-1 Grande
    The prompt consists of input-output pairs of examples ("few-shot"), where the last example only contain the input.
    """
    skills_str = ""
    for i, s in enumerate(skills, start=1):
        skills_str += f"{i:d}. {s}\n"
    skills_str += "\n"

    prompt = f"Write a winning CV profile for a Business Manager incorporating the following features:\n1. Logical mind\n2. Problem solver\n3. 2 years of experience in management\n4. Eager to learn\n\nProfile:\nI have a clear, logical mind with a practical approach to problem-solving and a drive to see things through to completion. I have more than 2 years of experience in managing and leading teams across multiple sectors. I am eager to learn, I enjoy overcoming challenges, and I have a genuine interest in Business Management and making organisations successful.\n\n##\n\nWrite a winning CV profile for a Chemical Engineer incorporating the following features:\n1. Hard worker and devoted\n2. Background in: design, plant operations, offshore operations, and process and safety improvements\n3. Experience in: designing, testing and analysing processes\n\nProfile:\nI am a dedicated, hardworking and proactive Chemical Engineer with a strong background in design, plant operations, offshore operations, and process and safety improvements. I have solid work experience in designing, testing and analysing processes to increase the overall efficiency of operations. I am currently looking for an opportunity to utilise my technical skills in a challenging working environment and become a valuable asset to the organisation that I work for.\n\n##\n\nWrite a winning CV profile for an IT Professional incorporating the following features:\n1. 5 years experience\n2. Record designing websites, networking and managing databases\n3. Excellent interpersonal skills\n4. Looking for a challenge\n\nProfile:\nI am a highly competent IT professional with 5 years of relevant industry experience and  a proven track record in designing websites, networking and managing databases. I have strong technical skills as well as excellent interpersonal skills, enabling me to interact with a wide range of clients. I am eager to be challenged in order to grow and further improve my IT skills. My greatest passion is in life is using my technical know-how to benefit other people and organisations.\n\n##\n\nWrite a winning CV profile for a {role} incorporating the following features:\n{skills_str}Profile:\n"
    return prompt

In [10]:
# Create a prompt for Sales Executive and look at the prompt
role = "Sales Executive"
features = ["Energetic and ambitious", "6 years of selling experience", "Independent", "MBA from Stanford"]

prompt = create_cv_profile_prompt(role, features)
print(prompt)

Write a winning CV profile for a Business Manager incorporating the following features:
1. Logical mind
2. Problem solver
3. 2 years of experience in management
4. Eager to learn

Profile:
I have a clear, logical mind with a practical approach to problem-solving and a drive to see things through to completion. I have more than 2 years of experience in managing and leading teams across multiple sectors. I am eager to learn, I enjoy overcoming challenges, and I have a genuine interest in Business Management and making organisations successful.

##

Write a winning CV profile for a Chemical Engineer incorporating the following features:
1. Hard worker and devoted
2. Background in: design, plant operations, offshore operations, and process and safety improvements
3. Experience in: designing, testing and analysing processes

Profile:
I am a dedicated, hardworking and proactive Chemical Engineer with a strong background in design, plant operations, offshore operations, and process and safety

### C. Perform real-time inference

Time to put Jurassic-1 Grande to work!

Let it generate a CV profile for the given input. Since the generation is not deterministic, you can ask for several completions per request and pick the one you prefer.

In [16]:
numResults = 3

response = ai21.Completion.execute(sm_endpoint="j1-grande",
                                   prompt=prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=['##'],
                                   numResults=numResults)

for i in range(numResults):
    print(response['completions'][i]['data']['text'].strip())
    print("=============")

I have an energetic and ambitious personality with a proven track record of selling products successfully. I have 6 years of selling experience, and 2 years of international experience. I am independent, resourceful, and tenacious. I have an MBA from Stanford. I am looking for a challenging position where I can utilise my skills and experience to meet my full potential.
I am an energetic and ambitious Sales Executive with 6 years of selling experience. I have outstanding interpersonal skills, a solid business acumen, and a proven track record of building and maintaining long-term relationships with customers. I have an MBA and I would like to work for a company that will allow me to use my skills and experience to further grow the business.
I’m an energetic and ambitious Sales Executive with 6 years of selling experience. I have excellent communication skills and thrive on meeting and interacting with people on a daily basis. I am an independent person who can deal with pressure and ma

### D. Create your own CV Profile!

In [17]:
role = input("Role:")
skills = input("Skills, separated by commas with no spaces: (for example, \'Hard worker,Resourceful\')")

Role: Developer Advocate
Skills, separated by commas with no spaces: (for example, 'Hard worker,Resourceful') Sassy,Loud,Character


In [18]:
# Create the prompt and look at it
your_prompt = create_cv_profile_prompt(role, skills.split(","))
print(your_prompt)

Write a winning CV profile for a Business Manager incorporating the following features:
1. Logical mind
2. Problem solver
3. 2 years of experience in management
4. Eager to learn

Profile:
I have a clear, logical mind with a practical approach to problem-solving and a drive to see things through to completion. I have more than 2 years of experience in managing and leading teams across multiple sectors. I am eager to learn, I enjoy overcoming challenges, and I have a genuine interest in Business Management and making organisations successful.

##

Write a winning CV profile for a Chemical Engineer incorporating the following features:
1. Hard worker and devoted
2. Background in: design, plant operations, offshore operations, and process and safety improvements
3. Experience in: designing, testing and analysing processes

Profile:
I am a dedicated, hardworking and proactive Chemical Engineer with a strong background in design, plant operations, offshore operations, and process and safety

In [19]:
numResults = 3

response = ai21.Completion.execute(sm_endpoint="j1-grande",
                                   prompt=your_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=['##'],
                                   numResults=numResults)

for i in range(numResults):
    print(response['completions'][i]['data']['text'].strip())
    print("=============")

I am sassy, loud and have character. I have been working as a developer advocate for the past 3 years for a company called Bosh. Prior to that I spent 2 years as a developer evangelist, and 3 years as a software engineer.
I am a dynamic, sassy, loud and character developer advocate with 5 years of relevant industry experience. I currently work as a developer advocate in a software startup. I have helped the company grow from zero to 500k users in 1.5 years. I am an enthusiastic programmer with 5 years of industry experience.
I possess a sassy, loud, and outgoing character, and am not afraid to speak my mind. I possess great leadership skills and am able to motivate a team to achieve a common goal. I have 2 years’ experience in developing software as well as 2 years’ of being a team leader. I am looking for a position that will allow me to fully utilise my skills and help me achieve my career aspirations.


### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [7]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

NameError: name 'model' is not defined

## 4. Clean-up

### A. Delete the model

In [8]:
model.delete_model()

NameError: name 'model' is not defined